In [345]:
import pandas as pd 
import numpy as np 
import pyodbc
from datetime import datetime, timedelta
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy
import nltk
import timeit
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
# gensim
from gensim import corpora, models, similarities, matutils
import unicodedata

from sklearn.feature_extraction.text import TfidfVectorizer
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ialzu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ialzu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [511]:
server = 't5.database.windows.net'
#server='192.168.0.100'
database = 'T5' 
username = 'T5' 
password = 'My404Data' 

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
df= pd.read_sql("select top(1000) * from texts",cnxn)
#df.to_csv("texts.csv",encoding='utf-8-sig')

In [347]:
df2= pd.read_sql("select  * from Articles",cnxn)
df2.to_csv("articles.csv",encoding='utf-8-sig')
df2

,A_ID,A_Link,A_LinkNum,A_Timestr,A_Cat,A_Auth,A_Title1,A_Title2,A_Paragraphs,A_Words,A_Characters
0,1,http://www.alriyadh.com/1814297,1814297,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,"الرياض - ""الرياض""",إلى ما بعد اجتماع مجموعة أوبك بلس,السعودية تؤجل الإعلان عن أسعار الخام لشهر مايو,8,339,1962
1,2,http://www.alriyadh.com/1814297,1814297,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,"الرياض - ""الرياض""",إلى ما بعد اجتماع مجموعة أوبك بلس,السعودية تؤجل الإعلان عن أسعار الخام لشهر مايو,8,339,1962
2,3,http://www.alriyadh.com/1814296,1814296,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,وادي الدواسر - سعود آل مسيب,,تعليم وادي الدواسر يبدأ المرحلة الثانية من خطة...,3,183,1162
3,4,http://www.alriyadh.com/1814296,1814296,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,وادي الدواسر - سعود آل مسيب,,تعليم وادي الدواسر يبدأ المرحلة الثانية من خطة...,3,183,1162
4,5,http://www.alriyadh.com/1814295,1814295,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,الرياض - محمد الحيدر,من برنامج إعانة الباحثين عن عمل,"""هدف"" يودع 446 مليون ريال في حسابات المستفيدين",4,190,1148
...,...,...,...,...,...,...,...,...,...,...,...
567519,198035,http://www.alriyadh.com/1706009,1706009,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,"جدة - "" مكتب جدة""",,"رئيس المساحة الجيولوجيه"" : اليوم الوطني ذكرى ن...",6,306,1862
567520,198036,http://www.alriyadh.com/1706008,1706008,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,"الرياض - ""الرياض""",,د.السبتي: اليوم الوطني يوم للوفاء والولاء و...,8,468,2887
567521,198037,http://www.alriyadh.com/1706008,1706008,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,"الرياض - ""الرياض""",,د.السبتي: اليوم الوطني يوم للوفاء والولاء و...,8,468,2887
567522,198038,http://www.alriyadh.com/1706007,1706007,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,الرياض- سعيد المبارك,,"نائب ""البيئة"" : 88 عاماً من العطاء والبناء",4,220,1369


In [449]:
df = pd.read_csv("texts.csv")
df

,Unnamed: 0,T_ID,T_AID,T_Text
0,0,87160,174309,\nعلى الرغم من التحذيرات الشديدة والمتكررة الت...
1,1,87164,174317,\nأدى محافظ ضمد عبدالله خالد البراق عقب صلاة ع...
2,2,87165,174319,\nرفع رئيس مجلس إدارة الهيئة العامة للولاية عل...
3,3,87168,174325,\nوجه صاحب السمو الملكي الأمير د. فيصل بن مشعل...
4,4,87169,174327,\nاستقبل صاحب السمو الملكي الأمير فيصل بن خالد...
...,...,...,...,...
995,995,87881,175751,"\nفازت ""هواوي""، الرائدة عالمياً في توفير البنى..."
996,996,190870,381731,\nأصدرت إدارة الخدمات الاجتماعية بوكالة الرئاس...
997,997,190872,381735,\nعشق الساحرة المستديرة دفع العديد من جماهير ك...
998,998,87882,175753,\nحذرت عميدة كلية الطب بجامعة الطائف د. دلال ن...


In [512]:
df3 = pd.merge(df2, df, how='inner', left_on='A_ID', right_on='T_AID')
df3

,A_ID,A_Link,A_LinkNum,A_Timestr,A_Cat,A_Auth,A_Title1,A_Title2,A_Paragraphs,A_Words,A_Characters,T_ID,T_AID,T_Text
0,462879,http://www.alriyadh.com/1557478,1557478,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,دنيا الرياضة,المدينة المنورة - خالد الحربي,,النصر وحيداً في صدارة «الناشئين»,2,87,506,231444,462879,\nانفرد النصر بصدارة الدوري الممتاز للناشئين ب...
1,462881,http://www.alriyadh.com/1557477,1557477,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,الكاريكاتير,,,ربيع,0,0,0,231445,462881,
2,462891,http://www.alriyadh.com/1557472,1557472,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,الأخــيــرة,الرياض - راشد السكران,إقرار كتاب مكافحة المخدرات كمقرر إجباري في الج...,ولي العهد يوجه بإدراج مفاهيم أضرار المخدرات ضم...,2,103,667,231450,462891,\nوجه صاحب السمو الملكي الأمير محمد بن نايف بن...
3,463567,http://www.alriyadh.com/1557107,1557107,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,الاقتصاد,واشنطن - د ب أ,,طائرة ترامب الرئاسية بأقل من أربعة مليارات دولار,2,124,747,231788,463567,\nناقش الرئيس الأمريكي المنتخب دونالد ترامب، و...
4,463561,http://www.alriyadh.com/1557110,1557110,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,الأولـــى,الأمير فيصل بن بندر يتسلم درعاً من وزير التعليم,دشّن مشروعات جامعة شقراء بتكلفة 500 مليون ريال,أمير الرياض: الميزانية مشجعة للوطن والمواطن,1,72,438,231785,463561,\nأكد صاحب السمو الملكي الأمير فيصل بن بندر أم...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,458821,http://www.alriyadh.com/1559680,1559680,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,الأولـــى,رجل يعبر بدراجته أمام ركام بنايات في دوما ...,,هدوء يسود معظم المناطق السورية ورصد عدة خروقات...,3,102,599,229415,458821,\nشهدت معظم المناطق السورية هدوءًا في اليوم ال...
996,459757,http://www.alriyadh.com/1559172,1559172,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,الداير - عافية الفيفي,,أهالي العزة ببني مالك يشيعون جثمان الشهيد شريف...,3,77,459,229883,459757,\nشيع أهالي جبل العزة بمحافظة الداير بني مالك ...
997,460557,http://www.alriyadh.com/1558747,1558747,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,متابعات,الأمير د. مشعل بن عبدالله,,د. مشعل بن عبدالله يوجه باستقبال تبرعات حملة ا...,1,106,672,230283,460557,\nوجه صاحب السمو الأمير د. مشعل بن عبدالله بن ...
998,461203,http://www.alriyadh.com/1558378,1558378,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,الدمام – محمد الغامدي,,وفاة طفل وإصابة 7 أشخاص إثر حريق اندلع بشقة في...,4,117,652,230606,461203,\nلقى طفل مصرعه وأصيب 5 نساء ورجلين في حريق شب...


In [513]:
df3['Words_Number'] = df3['T_Text'].astype(str).apply(lambda x : len(x.split()))
df3['T_Text'] = df3['T_Text'].apply(lambda x:unicodedata.normalize("NFKD", str(x)).replace('\n', ''))
# Wrong Texts (i.e. very long/short texts)
df3 = df3[((df3['Words_Number'] > 150) & (df3['Words_Number'] < 500))]
df3 = df3.reset_index()
#df3
Articles = list(df3['T_Text'].tolist())

In [514]:
wordtoken = word_tokenize(' '.join(Articles))

In [515]:
len(wordtoken)

91824

In [516]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words = stopwords.words('arabic'),token_pattern='\\b[\u0621-\u064A][\u0621-\u064A]+\\b')
#count_vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words = stopwords.words('arabic'),token_pattern='\\b[\u0621-\u064A][\u0621-\u064A]+\\b')
 
X = count_vectorizer.fit_transform(Articles)
cols = count_vectorizer.get_feature_names_out()



C:\Users\ialzu\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  warnings.warn(


In [517]:

#print(cols[100064])
#X.toarray()
# without stop words removal we got 138,561
# using arabic stop words we got 138,108
#print(X)
#print(X[1:1])

In [518]:
X.shape

(331, 74050)

In [519]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [520]:
doc_word = count_vectorizer.transform(Articles).transpose()

In [521]:
ab = pd.DataFrame(doc_word.toarray(), count_vectorizer.get_feature_names())
ab['Total'] = ab.sum(axis=1)
ab = ab.sort_values('Total',ascending=False)
ab.head(60)
#ab2
#ab2.shape
#ab2.max()
#ab2.argmax()
#ab[ab2.argmax():ab2.argmax()+1]

C:\Users\ialzu\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0,1,2,3,4,5,6,7,8,9,...,322,323,324,325,326,327,328,329,330,Total
الا,6,8,6,13,9,13,11,4,5,5,...,8,22,5,4,7,9,6,14,4,2756
لى,1,5,0,6,4,2,1,1,1,3,...,3,2,1,5,2,4,3,0,5,766
بن,4,5,1,6,0,0,1,5,3,0,...,1,1,0,0,0,1,0,0,0,566
المملكة,0,3,0,3,0,6,1,2,0,2,...,6,1,1,0,0,0,2,8,0,356
خلال,1,2,0,2,0,0,3,1,0,2,...,4,0,0,0,0,3,1,3,4,284
والا,0,0,3,0,4,3,2,1,1,0,...,0,0,1,0,1,1,3,2,4,277
الله,0,0,4,4,3,1,1,4,9,0,...,0,0,0,0,0,0,0,0,0,255
با,0,0,0,4,1,0,1,0,0,0,...,2,0,0,0,1,0,0,1,1,239
السعودية,0,0,0,0,0,1,0,0,0,2,...,8,1,0,0,0,0,0,4,0,238
مير,3,9,0,7,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,224


In [385]:
doc_word.shape

(74050, 331)

In [386]:
corpus = matutils.Sparse2Corpus(doc_word)
#models.LdaModel()

In [431]:
lda = models.LdaModel(corpus=corpus, num_topics=10, id2word=id2word, passes=3,iterations=100)

2021-11-09 22:30:34,136 : INFO : using symmetric alpha at 0.1
2021-11-09 22:30:34,136 : INFO : using symmetric eta at 0.1
2021-11-09 22:30:34,144 : INFO : using serial LDA version on this node
2021-11-09 22:30:34,208 : INFO : running online (multi-pass) LDA training, 10 topics, 3 passes over the supplied corpus of 331 documents, updating model once every 331 documents, evaluating perplexity every 331 documents, iterating 100x with a convergence threshold of 0.001000
2021-11-09 22:30:34,208 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2021-11-09 22:30:35,434 : INFO : -17.173 per-word bound, 147740.5 perplexity estimate based on a held-out corpus of 331 documents with 153319 words
2021-11-09 22:30:35,434 : INFO : PROGRESS: pass 0, at document #331/331
2021-11-09 22:30:36,026 : INFO : topic #7 (0.100): 0.009*"الا" + 0.004*"لى" + 0.001*"با" + 0.001*"المملكة" + 0.001*"بن" + 0.001*"خلال" + 0.001*"والا" + 

In [522]:
corpus = matutils.Sparse2Corpus(doc_word)
corpus

In [523]:
lda.print_topics()


2021-11-09 22:52:24,991 : INFO : topic #0 (0.100): 0.009*"الا" + 0.003*"بن" + 0.002*"لى" + 0.002*"الله" + 0.002*"يس" + 0.002*"المملكة" + 0.002*"السعودية" + 0.002*"عبدالعزيز" + 0.001*"العامة" + 0.001*"الهيي"
2021-11-09 22:52:25,015 : INFO : topic #1 (0.100): 0.010*"الا" + 0.004*"لى" + 0.003*"بن" + 0.002*"الله" + 0.001*"ية" + 0.001*"المملكة" + 0.001*"مير" + 0.001*"ول" + 0.001*"ال" + 0.001*"خلال"
2021-11-09 22:52:25,042 : INFO : topic #2 (0.100): 0.012*"الا" + 0.003*"لى" + 0.002*"بن" + 0.001*"للا" + 0.001*"تم" + 0.001*"المملكة" + 0.001*"العام" + 0.001*"العمل" + 0.001*"خلال" + 0.001*"دارة"
2021-11-09 22:52:25,052 : INFO : topic #3 (0.100): 0.020*"الا" + 0.005*"لى" + 0.002*"بن" + 0.002*"المملكة" + 0.001*"العام" + 0.001*"والا" + 0.001*"خلال" + 0.001*"با" + 0.001*"الله" + 0.001*"للا"
2021-11-09 22:52:25,058 : INFO : topic #4 (0.100): 0.012*"الا" + 0.003*"بن" + 0.002*"لى" + 0.002*"المملكة" + 0.002*"الله" + 0.002*"السعودية" + 0.002*"والا" + 0.001*"خلال" + 0.001*"مير" + 0.001*"الملك"
2021-11-09 

[(0,
  '0.009*"الا" + 0.003*"بن" + 0.002*"لى" + 0.002*"الله" + 0.002*"يس" + 0.002*"المملكة" + 0.002*"السعودية" + 0.002*"عبدالعزيز" + 0.001*"العامة" + 0.001*"الهيي"'),
 (1,
  '0.010*"الا" + 0.004*"لى" + 0.003*"بن" + 0.002*"الله" + 0.001*"ية" + 0.001*"المملكة" + 0.001*"مير" + 0.001*"ول" + 0.001*"ال" + 0.001*"خلال"'),
 (2,
  '0.012*"الا" + 0.003*"لى" + 0.002*"بن" + 0.001*"للا" + 0.001*"تم" + 0.001*"المملكة" + 0.001*"العام" + 0.001*"العمل" + 0.001*"خلال" + 0.001*"دارة"'),
 (3,
  '0.020*"الا" + 0.005*"لى" + 0.002*"بن" + 0.002*"المملكة" + 0.001*"العام" + 0.001*"والا" + 0.001*"خلال" + 0.001*"با" + 0.001*"الله" + 0.001*"للا"'),
 (4,
  '0.012*"الا" + 0.003*"بن" + 0.002*"لى" + 0.002*"المملكة" + 0.002*"الله" + 0.002*"السعودية" + 0.002*"والا" + 0.001*"خلال" + 0.001*"مير" + 0.001*"الملك"'),
 (5,
  '0.013*"الا" + 0.004*"لى" + 0.002*"خلال" + 0.002*"والا" + 0.002*"بن" + 0.001*"العمل" + 0.001*"با" + 0.001*"العامة" + 0.001*"العام" + 0.001*"المملكة"'),
 (6,
  '0.009*"الا" + 0.003*"لى" + 0.002*"المملكة" +

In [524]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]
#del lda_corpus

In [525]:
df[df['T_Text'].str.contains("ية ")].iloc[4].T_Text

'\nيفتتح خادم الحرمين الشريفين الملك سلمان بن عبدالعزيز آل سعود -حفظه الله- اليوم أَعْمَال السنة الثالثة من الدورة السابعة لمجلس الشورى، حيث يلقي -رَعَاهُ الله- خطاباً، يتضمن سياسة المملكة الدَّاخِلِيَّة والخارجية.\nويأتي الخطاب الملكي هذه المرة ومجلس الشورى على مقربة من الاحتفال بمرور مئة عام على تأسيس وإقرار أول مجلس شورى في تاريخ المملكة العربية السعودية، وذلك حسب ما نشرت مجلة المجلس في تقرير لها، موثقة التجربة الثرية والعميقة للمملكة، منذ أنّ اجتمع الملك المؤسس عبدالعزيز بن عبدالرحمن -طيب الله ثراه- في مكة المكرمة بعامة المواطنين وأعيان البلد من علماء ومشائخ وأمراء عشائر وتجار ورجال أعمال، وأطياف عديدة من المجتمع، ليسمع منهم ويلقي على مسامعهم خطابه الأول.\nورسم الملك المؤسس من خلال خطابه ملامح سياسته في إدارة الدولة، طالباً من الأهالي اختيار من يرون فيه الكفاءة والأهلية والقدرة على المساهمة في إدارة شؤون البلاد، والعمل على  إيصال ما يحتاجونه وما يتطلعون إليه كمواطنين، لتسهيل سير الأعمال وتحقيق الطلبات والاحتياجات الملحة، ولرفع ما يتفقون عليه لإقراره واعتماده من قبله -طيب الله ثراه-

In [526]:
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]

In [527]:
df6 = pd.DataFrame(columns=['ArticleID','TopicID','Perc'])
def func(x):
    #pass
    #print(x['index'])
    global df6
    for x1 in x['D']:
        #print(x1[1])
        #df6.iloc[-1] = [0,x1[0],x1[1]]
        df6 = df6.append(pd.DataFrame(columns=['ArticleID','TopicID','Perc'],data=[[x['index'],x1[0],x1[1]]]))
        #df6.index = df6.index+1

In [528]:
# Check out the document vectors in the topic space for the first 5 documents
lda_docs[0:50]
df5 = pd.DataFrame(columns = ["D"],data=zip(lda_docs))
#a = np.array(lda_docs[2])
#a[:,1].max()

#df5['Perc'] = df5['D'].apply(lambda x:list(zip(x))[0][0][1])
df5 = df5.reset_index()
df5.apply(lambda x:func(x),axis=1)
df5.head()
df6.head()

,ArticleID,TopicID,Perc
0,0,8,0.996973
0,1,5,0.998745
0,2,8,0.996889
0,3,9,0.998871
0,4,1,0.998539


In [529]:
#df3 = df3.reset_index()
df3.head()

,index,A_ID,A_Link,A_LinkNum,A_Timestr,A_Cat,A_Auth,A_Title1,A_Title2,A_Paragraphs,A_Words,A_Characters,T_ID,T_AID,T_Text,Words_Number
0,372,174325,http://www.alriyadh.com/1718982,1718982,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,الأمير د. فيصل بن مشعل,,تشكيل لجانٍ حصر أضرار السيول بالقصيم,3,165,1011,87168,174325,وجه صاحب السمو الملكي الأمير د. فيصل بن مشعل ...,165
1,378,174321,http://www.alriyadh.com/1718984,1718984,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,أمير الباحة مستقبلاً سفير السويد,,حسام بن سعود يطّلع على برنامج تطوير وزارة الدا...,12,414,2547,87166,174321,استقبل صاحب السمو الملكي الأمير د. حسام بن سع...,414
2,379,174387,http://www.alriyadh.com/1718950,1718950,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,متابعات,علوان بن سليمان الحربي,,الزيارة الكريمة,2,159,949,87199,174387,إنّ منطقة تبوك تتوشح جمالاً وتكتسي فرحاً وسرو...,159
3,380,174323,http://www.alriyadh.com/1718983,1718983,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,الأمير سعود بن نايف مستقبلاً وزير العمل,سعود بن نايف: العمل الخيري في المملكة يمضي على...,أمير الشرقية يطّلع على خطط العمل في توظيف الشباب,11,462,2840,87167,174323,استقبل صاحب السمو الملكي الأمير سعود بن نايف ...,462
4,383,174331,http://www.alriyadh.com/1718979,1718979,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,المريض تمكن من المشي بعد ساعة من الجراحة,,مستشفى د. سليمان الحبيب يستخدم ثاني أكسيد الكر...,5,377,2147,87171,174331,نجح الفريق الطبي بمركز طب وجراحة القلب بمستشفى...,377


In [530]:
#df6[df6['TopicID']==12]
df7 = pd.merge(df6,df3,how='left',left_on='ArticleID',right_on='index')
df7[df7['TopicID']==9][['ArticleID','TopicID','Perc','A_Cat','A_Title1','A_Title2','T_Text']]

#df7[df7['Index']==4][['ArticleID','TopicID','Perc','A_Cat','A_Title1','A_Title2','T_Text']]
Articles[4]

'نجح الفريق الطبي بمركز طب وجراحة القلب بمستشفى د. سليمان الحبيب بالريان، في إجراء جراحة معقدة لإصلاح الشريان الأورطي لستيني وإنقاذ حياته، وذلك بعد أن أصيب بتوسع في الشريان الأورطي حتى وصل إلى 12 سم، وإصابته بجلطة قلبية حادة.وقال رئيس الفريق الطبي المعالج استشاري أمراض القلب والأوعية الدموية والحاصل على البورد الأميركي د. عويّد الشمري، إن المريض نُقل لقسم الطوارئ وهو في حالة حرجة ويشكو من آلام شديدة في الظهر، وأجريت له الفحوصات اللازمة وتمت مراجعة تاريخه المرضي، وأظهرت النتائج إصابته بتضخم شديد جداً في الشريان الأورطي استمر معه لعدة سنوات حتى وصل حجمه إلى 12 سم بعد أن كان 8 سم في عام 2015م، وهو يعد من الحالات الخطرة جداً والنادرة، إذ إنه من المفترض أن لا يتجاوز عرض هذا الشريان 2.5 سم فقط وأن لا يكون تسارع التضخم أكثر من 3 ملم في العام، وإلا فإن احتمالية انفجار الشريان تصبح كبيرة، كما أن معدلات الوفاة نتيجة لانفجار الشريان قد تصل إلى 100 %.ونوّه د. الشمري أن هذا التضخم تسبب في الضغط على أعضاء البطن الحيوية خصوصاً الكلية اليمنى والعمود الفقري مما نتج عنه

In [429]:
Articles[100]

'تبدأ قناة «mbc 4» بعرض الدراما الاجتماعية البرتغالية، مسلسل «وردة من الماضي» والذي تدور أحداثة حول أختين تتحولاّن في لحظة إلى أعداء، فتقرّران الافتراق عن بعضهما وتأسيس كل منهما لحياة مستقلة، حيث يمضي الزمن وتلتقي الأختان في ظرفٍ استثنائي بعد سنوات طويلة، بعد أن تصاب «روزا» بحالة فقدان جزئي للذاكرة إثر تعرضها لحادث سيارة، فيُمحَى الحاضر واليوميات التي تعيشها مع زوجها وابنتيها، وتبقى ذاكرتها البعيدة حاضرة بأدقّ تفاصيلها، وما عاشته من وقت مع شقيقتها «نرجس» في فترة معيّنة من حياتهما معاً. فتواجهها المفاجآت والصدمات التي تظهر بين الحين والآخر وكيف تستغل نرجس الحادث لوضع يدها على كل الثروة التي جمعتها شقيقتها في السنوات الأخيرة. في هذه الأثناء، يشعر «دانيال» زوج روزا بغربة عن شريكة حياته بين ليلة وضحاها، فبعد أن كان يعتقد بأن زوجته هي ابنة وحيدة لوالديها يكتشف فجأة، وبعد عشر سنوات خبايا لم يكن يعرفها عن زوجته، خصوصاً بعدما أفقدتها الحادثة ذاكرتها القريبة، لتحتفظ فقط بذاكرتها البعيدة، بحيث لم تعد تتعرّف على أحد سوى أختها.يذكر أن مسلسل «وردة من الماضي» يعرض على قناة «mb

In [441]:
x=lda.show_topics(num_topics=12, num_words=5,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Topics and Words
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()

#Below Code Prints Only Words 
for topic,words in topics_words:
    print(" ".join(words))

0::['الا', 'بن', 'لى', 'الله', 'يس']
1::['الا', 'لى', 'بن', 'الله', 'ية']
2::['الا', 'لى', 'بن', 'للا', 'تم']
3::['الا', 'لى', 'بن', 'المملكة', 'العام']
4::['الا', 'بن', 'لى', 'المملكة', 'الله']
5::['الا', 'لى', 'خلال', 'والا', 'بن']
6::['الا', 'لى', 'المملكة', 'بن', 'السعودية']
7::['الا', 'لى', 'نقاط', 'الترتيب', 'تا']
8::['الا', 'بن', 'لى', 'الملك', 'المملكة']
9::['الا', 'بن', 'لى', 'مير', 'المملكة']

الا بن لى الله يس
الا لى بن الله ية
الا لى بن للا تم
الا لى بن المملكة العام
الا بن لى المملكة الله
الا لى خلال والا بن
الا لى المملكة بن السعودية
الا لى نقاط الترتيب تا
الا بن لى الملك المملكة
الا بن لى مير المملكة
